# _STAT 201 Group Project Proposal_

##### _Group 31: Alexandr Kim, Yingshan Wu, Yang Lei, Harbor Zhang_

## Introduction

## Preliminary Results

## Methods: Plan

### Adding Necessary Packages

In order to load the packages, we have to use the code `install.packages()`

In [5]:
install.packages("tidyverse")
install.packages("tidymodels")
install.packages("repr")
install.packages("cowplot")
install.packages("ggplot2")
install.packages("GGally")
install.packages("digest")
install.packages("infer")
install.packages("RCurl")

also installing the dependencies ‘rappdirs’, ‘gargle’, ‘ids’, ‘dtplyr’, ‘googledrive’, ‘googlesheets4’, ‘lubridate’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependencies ‘warp’, ‘patchwork’, ‘slider’, ‘conflicted’, ‘infer’, ‘modeldata’, ‘recipes’, ‘rsample’, ‘tune’, ‘workflowsets’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in install.packages("cowplot"):
“installation of package ‘cowplot’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Warning message in install.packages("GGally"):
“installation of package ‘GGally’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages i

After installing relevant packages, we need to load it into workflow using `library()`

In [1]:
library(tidyverse)
library(cowplot)
library(infer)
library(repr)
library(ggplot2)
library(XML)
library(RCurl)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



ERROR: Error in library(XML): there is no package called ‘XML’


### Appendix A: Data set for street trees in Vancouver (BC, Canada) recorded by City of Vancouver

In [11]:
install.packages("data.table")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [12]:
library(data.table)


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [13]:
street_trees_data <- fread("https://opendata.vancouver.ca/explore/dataset/street-trees/table/?disjunctive.species_name&disjunctive.common_name&disjunctive.on_street&disjunctive.neighbourhood_name&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IkNPVU5UIiwieUF4aXMiOiJoZWlnaHRfcmFuZ2VfaWQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjMDI3OUIxIn1dLCJ4QXhpcyI6ImRhdGVfcGxhbnRlZCIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6InllYXIiLCJzb3J0IjoiIiwiY29uZmlnIjp7ImRhdGFzZXQiOiJzdHJlZXQtdHJlZXMiLCJvcHRpb25zIjp7ImRpc2p1bmN0aXZlLnNwZWNpZXNfbmFtZSI6dHJ1ZSwiZGlzanVuY3RpdmUuY29tbW9uX25hbWUiOnRydWUsImRpc2p1bmN0aXZlLm9uX3N0cmVldCI6dHJ1ZSwiZGlzanVuY3RpdmUubmVpZ2hib3VyaG9vZF9uYW1lIjp0cnVlfX19XSwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D")

head(street_trees_data)

Warning message in fread("https://opendata.vancouver.ca/explore/dataset/street-trees/table/?disjunctive.species_name&disjunctive.common_name&disjunctive.on_street&disjunctive.neighbourhood_name&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IkNPVU5UIiwieUF4aXMiOiJoZWlnaHRfcmFuZ2VfaWQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjMDI3OUIxIn1dLCJ4QXhpcyI6ImRhdGVfcGxhbnRlZCIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6InllYXIiLCJzb3J0IjoiIiwiY29uZmlnIjp7ImRhdGFzZXQiOiJzdHJlZXQtdHJlZXMiLCJvcHRpb25zIjp7ImRpc2p1bmN0aXZlLnNwZWNpZXNfbmFtZSI6dHJ1ZSwiZGlzanVuY3RpdmUuY29tbW9uX25hbWUiOnRydWUsImRpc2p1bmN0aXZlLm9uX3N0cmVldCI6dHJ1ZSwiZGlzanVuY3RpdmUubmVpZ2hib3VyaG9vZF9uYW1lIjp0cnVlfX19XSwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D"):
“Stopped early on line 72. Expected 5 fields but found 0. Consider fill=TRUE and comment.char=. First discarded non-empty line: <<<base href="/explore/dataset/street-trees/">>>”


"<link rel=""alternate"" type=""application/rss+xml"" title=""Street trees"" href=""http://opendata.vancouver.ca/explore/dataset/street-trees/rss/?disjunctive.species_name=&amp",disjunctive.common_name=&amp,disjunctive.on_street=&amp,disjunctive.neighbourhood_name=&amp,"dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IkNPVU5UIiwieUF4aXMiOiJoZWlnaHRfcmFuZ2VfaWQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjMDI3OUIxIn1dLCJ4QXhpcyI6ImRhdGVfcGxhbnRlZCIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6InllYXIiLCJzb3J0IjoiIiwiY29uZmlnIjp7ImRhdGFzZXQiOiJzdHJlZXQtdHJlZXMiLCJvcHRpb25zIjp7ImRpc2p1bmN0aXZlLnNwZWNpZXNfbmFtZSI6dHJ1ZSwiZGlzanVuY3RpdmUuY29tbW9uX25hbWUiOnRydWUsImRpc2p1bmN0aXZlLm9uX3N0cmVldCI6dHJ1ZSwiZGlzanVuY3RpdmUubmVpZ2hib3VyaG9vZF9uYW1lIjp0cnVlfX19XSwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D"" />"
<chr>,<chr>,<chr>,<chr>,<chr>
"<link rel=""alternate"" type=""application/atom+xml"" title=""Street trees"" href=""http://opendata.vancouver.ca/explore/dataset/street-trees/atom/?disjunctive.species_name=&amp",disjunctive.common_name=&amp,disjunctive.on_street=&amp,disjunctive.neighbourhood_name=&amp,"dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJsaW5lIiwiZnVuYyI6IkNPVU5UIiwieUF4aXMiOiJoZWlnaHRfcmFuZ2VfaWQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjMDI3OUIxIn1dLCJ4QXhpcyI6ImRhdGVfcGxhbnRlZCIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6InllYXIiLCJzb3J0IjoiIiwiY29uZmlnIjp7ImRhdGFzZXQiOiJzdHJlZXQtdHJlZXMiLCJvcHRpb25zIjp7ImRpc2p1bmN0aXZlLnNwZWNpZXNfbmFtZSI6dHJ1ZSwiZGlzanVuY3RpdmUuY29tbW9uX25hbWUiOnRydWUsImRpc2p1bmN0aXZlLm9uX3N0cmVldCI6dHJ1ZSwiZGlzanVuY3RpdmUubmVpZ2hib3VyaG9vZF9uYW1lIjp0cnVlfX19XSwiZGlzcGxheUxlZ2VuZCI6dHJ1ZSwiYWxpZ25Nb250aCI6dHJ1ZX0%3D"" />"
